In [ ]:
# Given the `test1.csv` dataset, predict the salary of an employee based
# on their age and experience

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [12]:
# Read the dataset
training = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [13]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [14]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [15]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [16]:
# Group independent variables into vectors
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent Features')

In [17]:
output = featureassembler.transform(training)

In [18]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [19]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [20]:
finalized_data = output.select('Independent Features', 'Salary')

In [21]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
# Train test split
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor =  LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

24/06/04 01:51:18 WARN Instrumentation: [c0393466] regParam is zero, which might cause numerical instability and overfitting.
24/06/04 01:51:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/04 01:51:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/06/04 01:51:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [24]:
# Coefficients
regressor.coefficients

DenseVector([-500.0, 2500.0])

In [26]:
# Intercepts
regressor.intercept

24499.999999998796

In [27]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [28]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16500.000000000153|
|          [30.0,8.0]| 25000|29499.999999999152|
|         [31.0,10.0]| 30000| 33999.99999999875|
+--------------------+------+------------------+



In [29]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(3333.3333333326846, 12833333.333327604)